<a href="https://colab.research.google.com/github/MatthiasReccius/kommunal-o-mat/blob/main/quickstarts/rag_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Install SDK

Install the SDK from [PyPI](https://github.com/googleapis/python-genai).

In [1]:
%pip install -q -U "google-genai>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.7/241.7 kB 6.4 MB/s eta 0:00:00


### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key or you aren't sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

### Initialize SDK client and mount drive

With the new SDK, now you only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://cloud.google.com/vertex-ai)). The model is now set in each call.

In [2]:
from google import genai
from google.colab import userdata, drive

drive.mount('/content/drive', force_remount=True)
client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

Mounted at /content/drive


## Import packages

In [3]:
import json
import pandas as pd
from pathlib import Path
from google.genai.types import EmbedContentConfig
from tqdm import tqdm
import time
import random
import hashlib
import numpy as np
import re

196

In [31]:
out_dir = Path("/content/drive/MyDrive/kommunal-o-mat/")

### Import and clean data

In [5]:
jsonl_path = Path('/content/drive/MyDrive/kommunal-o-mat/program_segments.jsonl')

rows = [json.loads(line) for line in jsonl_path.read_text(encoding='utf-8').splitlines()]
df = pd.DataFrame(rows)  # columns: party, section, text
df["text"] = df["text"].str.replace("\n"," ")
len(df)

196

In [6]:
df

,party,section,text
0,AfD,"Umwelt, Klima und Energie",„Bürgerschutz statt teure Klimahysterie“ Es is...
1,AfD,"Umwelt, Klima und Energie",Für eine zukunftsfähige Personalpolitik“ Bund ...
2,AfD,"Umwelt, Klima und Energie",„Neutrale Schule ohne Extremismus“ In Dortmund...
3,AfD,"Umwelt, Klima und Energie",„Kultur vor Missbrauch durch Politpropaganda s...
4,AfD,"Umwelt, Klima und Energie",„Bürgerbeteiligung durch Bürgergutachten stärk...
...,...,...,...
191,SPD,"Umwelt, Klima und Energie",Durch Verfahrensvereinfachungen und eine maßvo...
192,SPD,"Umwelt, Klima und Energie","Wir wollen, dass Mieter*innen noch besser gege..."
193,SPD,"Umwelt, Klima und Energie",Berufstätige Familien stehen im Mittelpunkt so...
194,SPD,"Umwelt, Klima und Energie","Die Kita ist die erste Bildungsinstitution, di..."


### Choose a model

Select the model you want to use in this guide. You can either select one from the list or enter a model name manually.

For a full overview of all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini).

In [7]:
ENCODER_ID = "gemini-embedding-001" # @param ["gemini-embedding-001", "text-embedding-004"] {"allow-input":true, isTemplate: true}

## Batch embed content

You can embed a list of multiple prompts with one API call for efficiency.

In [10]:
def embed_batch(texts):
    # simple sequential batching to be gentle on rate limits
    out = []
    for t in texts:
        emb = client.models.embed_content(
            model=ENCODER_ID,
            contents=t,
            config=EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT")
        ).embeddings[0].values
        out.append(emb)
    return out

In [28]:
def embed_with_retry(text, task="RETRIEVAL_DOCUMENT", max_retries=6, base=10.0):
    for attempt in range(max_retries):
        try:
            r = client.models.embed_content(
                model=ENCODER_ID,
                contents=text,
                config=EmbedContentConfig(task_type=task)
            )
            return r.embeddings[0].values
        except Exception as e:
            error_msg = str(e)
            if "RESOURCE_EXHAUSTED" in error_msg or "429" in error_msg:
                sleep = base * (2 ** attempt) + random.random()  # exp backoff + jitter
                print(f"Rate limit exceeded. Sleeping for {round(sleep)} seconds.")
                time.sleep(sleep)
                continue
            raise
    raise RuntimeError("Exceeded retries due to rate limits.")

In [29]:
calls_per_sec = 1
interval = 1.0 / calls_per_sec
last = 0.0

embs = []
for text in tqdm(df["text"]):
    now = time.time()
    if now - last < interval:
        time.sleep(interval - (now - last))
    embs.append(embed_with_retry(text, task="RETRIEVAL_DOCUMENT"))
    last = time.time()

df["embedding"] = embs

 28%|██▊       | 54/196 [00:36<01:34,  1.51it/s]

Rate limit exceeded. Sleeping for 10.0 seconds.
Rate limit exceeded. Sleeping for 10.372691215499987 seconds.
Rate limit exceeded. Sleeping for 11.04912151796734 seconds.


 41%|████▏     | 81/196 [01:27<01:25,  1.34it/s]

Rate limit exceeded. Sleeping for 10.0 seconds.
Rate limit exceeded. Sleeping for 18.315804462391963 seconds.
Rate limit exceeded. Sleeping for 19.887697148628515 seconds.


 47%|████▋     | 92/196 [02:23<01:58,  1.14s/it]

...

 60%|█████▉    | 117/196 [03:35<00:54,  1.44it/s]

Rate limit exceeded. Sleeping for 10.0 seconds.
Rate limit exceeded. Sleeping for 11.916508735136661 seconds.
Rate limit exceeded. Sleeping for 23.90396771869166 seconds.


 80%|████████  | 157/196 [04:49<00:26,  1.49it/s]

Rate limit exceeded. Sleeping for 10.0 seconds.


100%|██████████| 196/196 [05:25<00:00,  1.66s/it]


In [33]:
out_path = out_dir / "party_embeddings.parquet"
# Store df containing text chunks and embeddings
df.to_parquet(out_path, index=False)

In [23]:
df["embeddings"] = df.apply(lambda x: client.models.embed_content(model=ENCODER_ID, contents=(x['text']), config=EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT")).embeddings[0].values, axis=1)
df

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}

# Try the model

Now you will create a function to do the interaction between questions and the search in the dataframe.

The `find_best_passage` function to, instead of searching for simple keywords, it searches for meaning.

Here’s a step-by-step breakdown of what it does when you ask a question:

- First, the function takes your query (e.g., "how do I change gears?") and uses the embedding model to convert it into an embedding using the `RETRIEVAL_QUERY` task type.
- Then the function compares the embeddings from your question to the numbers of every single document. It calculates a similarity score for each pair. A higher score means the meanings are more closely aligned.
- Finally, the function identifies the document with the single highest similarity score and returns its original text as the most relevant answer to your question

In [34]:
def find_best_passage(query: str, dataframe: pd.DataFrame, model: str) -> str:

  # 1. Create an embedding for the user's query.
  query_embedding = client.models.embed_content(
      model=model,
      contents=query,
      config=EmbedContentConfig(task_type="RETRIEVAL_QUERY")
  )

  # 2. Calculate the dot product to find the similarity between the query and all documents
  dot_products = np.dot(np.stack(dataframe.embedding), query_embedding.embeddings[0].values)

  # 3. Find the index of the highest score and return the corresponding text.
  best_passage_index = np.argmax(dot_products)

  # 4. return the document contents more relevant to the question
  return dataframe.text.iloc[best_passage_index]

In [36]:
question = """Migration ist wichtig."""

best_passage = find_best_passage(question, df, ENCODER_ID)
best_passage

'Die Migration ist eine der großen Herausforderungen, kann aber auch eine große Chance für die Zukunft sein. Der Zuzug vieler Menschen in den vergangenen Jahren stellt unsere Stadt gerade im Bereich der schulischen und der Kita-Versorgung sowie auf dem Wohnungsmarkt vor erhebliche Herausforderungen. Eine dauerhafte Aufgabe ist es auch, das Entstehen von Parallelgesellschaften zu verhindern. Gleichzeitig benötigt unsere Gesellschaft viele qualifizierte Arbeitskräfte, was gezielte Zuwanderung aus dem Ausland notwendig macht. Dieser Zukunftsaufgabe stellt sich die CDU, ohne Missstände zu ignorieren oder Vorurteile zu bedienen. Wir wollen daher: Regeln durchsetzen Migration kann nur mit Regeln funktionieren. Daher wollen wir einen beliebigen Zuzug in unsere Gesellschaft nicht hinnehmen - eine Geltung des Rechts und der Grundwerte unserer Gesellschaft sind für uns selbstverständlich. Konkret fordern wir: •\tGute personelle Ausstattung der Ausländerbehörde, um aufenthaltsrechtliche Verwaltun

Now you can do one augmented generation (the last step of the RAG process) using the best passage found by the first step, but still having custom answers for users instead of simply pasting large documents chunks directly:

In [40]:
question = """Ausländer sind wichtig."""

In [42]:
from IPython.display import Markdown

final_answer_prompt = f"""Your Role: You are a friendly AI assistant. Your purpose is to retrieve relevant information from party programs and enable the user to compare and contrast the parties views on policy issues.

Your Task: Use the provided "Source Text" below to answer the user's question.

Guidelines for your Response:

Always answer in German.

Be Faithful: Always represent the party programs exactly as written. Quote directly when possible and avoid paraphrasing that could distort meaning.

Be Transparent: Always mention the field "party" in your answer to source the policy position.

Be Neutral: Present the programs in a balanced and impartial way, without adding interpretation, judgment, or outside commentary.

Be Thorough: Include all relevant positions, demands, or proposals from the provided text. If comparing parties, highlight similarities and differences strictly based on what is explicitly stated.

Stay on Source: Only use the information contained in the provided passage. If a specific position is not mentioned in the text, explicitly state that it is not available in the provided material. Do not use outside knowledge.

For every question, find the most fitting passage from EACH party. Always provide your answer according to the following structure:

Grüne:

QUESTION: {question}
PASSAGE: {best_passage}
"""

MODEL_ID = "gemini-2.5-flash"

final_answer = client.models.generate_content(
    model=MODEL_ID,
    contents=final_answer_prompt,
)

Markdown(final_answer.text)

**AfD:**

**QUESTION:** Ausländer sind wichtig.

**PASSAGE:** „Dortmund - sichere Heimat statt sicherer Hafen“
Für den linken multikulturellen Fiebertraum einer verkraftbaren Armutsmassenzuwanderung aus aller Welt dürfte Dortmund im letzten Jahrzehnt hunderte Millionen Euro verjubelt haben. Der Sozialstaat wird ausgehöhlt und beraubt sich seiner eigenen Grundlage - dem Sinn für Solidarität und Vertrauen in den Mitbürger. „Diversität“ zersetzt Ordnung und Vertrauen, wie selbst eine Studie der linksliberal geprägten Universität Melbourne feststellen musste. Je „bunter“ ein Stadtviertel, desto brüchiger der soziale Zusammenhalt.
Den „Heimathafen“ im Dortmunder Norden, der uns als „Prachtbau der Willkommenskultur“ mit fast 10 Millionen Euro in Rechnung gestellt wurde, wollen wir zu einem „Remigrationshafen“ umwidmen. Er ist Teil unserer Abschiebeagenda 2025/2035 im Rahmen unseres „Masterplans Remigration“ zur Rückführung illegaler sowie ausreisepflichtiger Ausländer. Bis 2035 streben wir in Dortmund Asylbetrugsneutralität an.
In Dortmund leben rund 2000 ausreisepflichtige Ausländer, die von der Stadt geduldet werden. In der Vergangenheit hat sich gezeigt, dass Ausländer ohne Bleibeperspektive immer wieder zum Sicherheitsrisiko erwachsen. Kriminelle Ausländer sind zuvorderst rückzuführen.
Unsere Ausländerbehörde muss endlich „Ausreisebehörde“ werden und darf nicht länger die fatale Willkommenspolitik mittragen und umsetzen. Eine weitere Aufnahme von Migranten im Rahmen des Projektes „Seenotbrücke“ lehnt die AfD Dortmund ab. Die bisherige Duldung ausreisepflichtiger Ausländer durch die Stadt ist zu beenden. Bis zu ihrer Abschiebung sind ausreisepflichtigen Ausländern im Rahmen der Gesetze Sach- statt Geldleistungen durch die Stadt zu gewähren. Sie sind zentral und vor allen Dingen nicht auf dem freien Wohnungsmarkt unterzubringen.
Unsere Kultur schwindet und unser Schuldenberg wächst maßgeblich auch durch die Ausgaben für Migration und Asyl unkontrolliert weiter. Zwischen 2018 und 2022 musste Dortmund allein für vollziehbar ausreisepflichtige Migranten rund 72,2 Mio. Euro aufbringen. Rechnen wir die Jahre 2023 und 2024 hinzu, kommen wir zweifellos auf einen dreistelligen Millionenbetrag an Ausgaben, die durch eine konsequente Abschiebepolitik von Land und Stadt vermeidbar gewesen wären.
Unsere Bürger zahlen auch den Lebensunterhalt für rund 50.000 ausländische Sozialhilfeempfänger, die in Dortmund mittlerweile rund 40 Prozent der Leistungsbezieher ausmachen. Darunter sind allein 4600 südosteuropäische Zuwanderer, die manchmal trotz zehnjährigem Aufenthalt immer noch keiner offiziellen Beschäftigung nachgehen.

# Learning more

Check out these examples in the Cookbook to learn more about what you can do with embeddings:

- [Search Reranking](../examples/Search_reranking_using_embeddings.ipynb): Use embeddings from the Gemini API to rerank search results from Wikipedia.
- [Anomaly detection with embeddings](../examples/anomaly_detection.ipynb): Use embeddings from the Gemini API to detect potential outliers in your dataset.
- [Train a text classifier](../examples/Classify_text_with_embeddings.ipynb): Use embeddings from the Gemini API to train a model that can classify different types of newsgroup posts based on the topic.

Embeddings have many applications in Vector Databases, too. Check out these examples:
- With [Chroma DB](../examples/chromadb)
- With [LangChain](../examples/langchain)
- With [LlamaIndex](../examples/llamaindex)
- With [Qdrant](../examples/qdrant)
- With [Weaviate](../examples/weaviate)

You can learn more about embeddings in general on ai.google.dev in the embeddings guide
